In [0]:
!unzip train-images-idx3-ubyte.zip
!unzip train-labels-idx1-ubyte.zip
!unzip t10k-images-idx3-ubyte.zip
!unzip t10k-labels-idx1-ubyte.zip

Archive:  train-images-idx3-ubyte.zip
  inflating: s118qIcfjlPm            
Archive:  train-labels-idx1-ubyte.zip
  inflating: s118JTYVl2zS            
Archive:  t10k-images-idx3-ubyte.zip
  inflating: s1187X8Z05ne            
Archive:  t10k-labels-idx1-ubyte.zip
  inflating: s118kEUna91h            


In [0]:
pip install idx2numpy

  Created wheel for idx2numpy: filename=idx2numpy-1.2.2-cp36-none-any.whl size=8032 sha256=c2b37e7feca90bb9269006d94dea04151b5e2f94dfb04669a804c45788c26b0d
  Stored in directory: /root/.cache/pip/wheels/7a/b5/69/3e0757b3086607e95db70661798fdf98a77a0bb79c54e1f320
Successfully built idx2numpy


In [0]:
#https://stackoverflow.com/questions/40427435/extract-images-from-idx3-ubyte-file-or-gzip-via-python
import idx2numpy
import numpy as np
file= 's118qIcfjlPm'
arr = idx2numpy.convert_from_file(file)
# arr is now a np.ndarray type of object of shape 60000, 28, 28
file='s118JTYVl2zS'
labels=idx2numpy.convert_from_file(file)
file='s1187X8Z05ne'
arr1=idx2numpy.convert_from_file(file)
file='s118kEUna91h'
test_labels=idx2numpy.convert_from_file(file)

In [0]:
data=[]
for i in range(0,len(arr)):
  a=arr[i].flatten()
  data.append(a)

In [0]:
test_data=[]
for i in range(0,len(arr1)):
  a=arr1[i].flatten()
  test_data.append(a)

In [0]:
test_data=np.asarray(test_data)

In [0]:
data=np.asarray(data)

In [0]:
#https://www.geeksforgeeks.org/numpy-mean-in-python/
global_mean=np.mean(data, axis = 0)

In [0]:
centralised_data=data-global_mean

In [0]:
#https://www.geeksforgeeks.org/python-numpy-cov-function/
global_cov_mat = np.stack(centralised_data, axis = 1)  
print("shape of covariance matrix:", np.shape(np.cov(global_cov_mat))) 
global_cov_mat=np.cov(global_cov_mat)

shape of covariance matrix: (784, 784)


In [0]:
#https://machinelearningmastery.com/calculate-principal-component-analysis-scratch-python/
from numpy.linalg import eigh
val, vector=eigh(global_cov_mat)

In [0]:
#print(centralised_data.shape)
projection=vector.T.dot(centralised_data.T)
#print(projection.shape)
projection_test=vector.T.dot(test_data.T)

In [0]:
a=int(input("enter number of dimensions for pca "))
proj_pca=[]
proj_pca_test=[]
for k in range(a):
  proj_pca.append(projection[-(k+1)])
  proj_pca_test.append(projection_test[-(k+1)])
  
proj_pca=np.asarray(proj_pca)
proj_pca_test=np.asarray(proj_pca_test)

enter number of dimensions for pca 600


In [0]:
proj_pca=proj_pca.T
proj_pca_test=proj_pca_test.T

Now applying FDA

In [0]:
class_name=set(labels)

In [0]:
print("Number of classes",class_name)

Number of classes {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [0]:
data1=[]
for i in range(len(class_name)):
  data1.append([])
#data1 contains data according to class label
for k in range(len(data)):
  data1[labels[k]].append(proj_pca[k])

In [0]:
data1=np.asarray(data1)

In [0]:
for i in range(len(class_name)):
  data1[i]=np.asarray(data1[i])


In [0]:
mean_class=[]
for i in range(len(class_name)):
  mean_class.append(np.mean(data1[i], axis = 0))

In [0]:
mean_class=np.asarray(mean_class)

In [0]:
#conversion to centralised data
centralised_data_class=[]

for i in range(len(class_name)):
  centralised_data_class.append(data1[i]-mean_class[i])     

In [0]:
centralised_data_class=np.asarray(centralised_data_class)

In [0]:
cov_mat_class=[]
for i in range(len(class_name)):
  cov_mat_class.append(np.cov(centralised_data_class[i].T))

In [0]:
cov_mat_class=np.asarray(cov_mat_class)

In [0]:
Sw=np.zeros(shape=(a,a))
for i in range(len(class_name)):
  Sw=Sw+cov_mat_class[i]

In [0]:
global_cov_mat_pca = np.stack(proj_pca, axis = 1)  
#print("shape of covariance matrix:", np.shape(np.cov(global_cov_mat))) 
global_cov_mat_pca=np.cov(global_cov_mat_pca)

In [0]:
Sb=global_cov_mat_pca-Sw

In [0]:
from numpy.linalg import pinv
inverse_Sw=pinv(Sw)

In [0]:
factor=inverse_Sw.dot(Sb)

In [0]:
from numpy.linalg import eigh
val, vector=eigh(factor)

In [0]:
projection_fda=vector.T.dot(proj_pca.T)
projection_fda_test=vector.T.dot(proj_pca_test.T)

In [0]:
b=int(input("Enter number of dimensions for FDA "))
proj_fda=[]
proj_fda_test=[]
for i in range(b):
  proj_fda.append(projection_fda[-(i+1)])
  proj_fda_test.append(projection_fda_test[-(i+1)])


Enter number of dimensions for FDA 300


In [0]:
proj_fda=np.asarray(proj_fda)
proj_fda_test=np.asarray(proj_fda_test)

In [0]:
proj_fda=proj_fda.T
proj_fda_test=proj_fda_test.T

applying LDA

In [0]:
def LDA(x,mean_class,inverse_cov_matrix,prior):
  g=[]
  for i in range(10):
    z=x-mean_class[i]
    #print(z.shape)
    B=np.matmul(z,inverse_cov_matrix)
    #print(B.shape)
    A=np.matmul(B,z.T)
    #print(A.shape)
    g.append(-1/2*A+np.log(prior[i]))
  a=g.index(max(g))
  return a

In [0]:
data1=[]
for i in range(len(class_name)):
  data1.append([])
#data1 contains data according to class label
for l in range(len(proj_fda)):
  data1[labels[l]].append(proj_fda[l])
data1=np.asarray(data1)
for i in range(len(class_name)):
  data1[i]=np.asarray(data1[i])
mean_class=[]
for i in range(len(class_name)):
  mean_class.append(np.mean(data1[i], axis = 0))
mean_class=np.asarray(mean_class)
#conversion to centralised data
centralised_data_class=[]

for i in range(len(class_name)):
  centralised_data_class.append(data1[i]-mean_class[i]) 
centralised_data_class=np.asarray(centralised_data_class) 
cov_mat_class=[]
for i in range(len(class_name)):
  cov_mat_class.append(np.cov(centralised_data_class[i].T))   
cov_mat_class=np.asarray(cov_mat_class)
prior=[]
for i in range(10):
  prior.append(len(data1[i])/len(data))
weighted_cov=np.zeros(shape=(b,b))
for i in range(len(class_name)):
  weighted_cov=weighted_cov+prior[i]*cov_mat_class[i]
#print((prior[i]*cov_mat_class[i]).shape)
from numpy.linalg import pinv
inverse_cov_matrix=pinv(weighted_cov)
  
  
count=0
for i in range(len(proj_fda_test)):
  #print(len(test_data[i]))
  res=LDA(proj_fda_test[i],mean_class,inverse_cov_matrix,prior)
  if(res==test_labels[i]):
    count=count+1
acc=(count/len(test_labels))*100
print("Accuracy is ",acc)

Accuracy is  82.1
